In [1]:
import numpy as np
from glob import glob
import pandas as pd
import random
from tqdm import tqdm
from sklearn import metrics
from sklearn.metrics import auc
import pickle
import pyod
from pyod.utils.stat_models import pairwise_distances_no_broadcast
import warnings
from icecream import ic
from src.constant import translation_dict
from torch.utils.data import random_split, DataLoader
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

%load_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")

In [2]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:15000")
mlflow.set_experiment("my-experiment")

<Experiment: artifact_location='s3://mlflow/1', creation_time=1716083738206, experiment_id='1', last_update_time=1716083738206, lifecycle_stage='active', name='my-experiment', tags={}>

In [3]:
import pandas as pd
import glob

# 获取所有的CSV文件
csv_path = 'data/Train'
pkl_path = 'data/pkl/Train'
csv_files = glob.glob(csv_path+'/*.csv')
pkl_files = glob.glob(pkl_path+'/*.pkl')

In [4]:
df_list = [pd.read_csv(f).replace({'label': {10: 1}}).assign(sn=i) for i, f in enumerate(csv_files)]
df_list = pd.concat(df_list)
df_list.shape


(7267584, 10)

In [5]:
from src.statistical_analyzer import StatisticalAnalyzer


StatisticalAnalyzer.show_report(df_list, 'data/report/report.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/Users/acke/Library/Caches/pypoetry/virtualenvs/battery-j7E_kv4E-py3.10/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'cannot reindex on an axis with duplicate labels')
  warnings.warn(


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# 数据展示

In [6]:
display_df = df_list[df_list["sn"]==df_list["sn"].unique()[1]]
# display_df.head().to_csv('data/output.csv', index=False)

In [9]:
display_df.to_csv('data/display.csv', index=False)

In [10]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go

# 自定义颜色列表
custom_colors = [
    "#7ec2f3",
    "#76d0a3",
    "#f79992",
    "#fa90ba",
    "#faaf76",
    "#b3acf2",
    "#ffdd76",
    "#76cdd3",
    "#e9e9e9",
]

def plot_lines(df,x_col,width=800,height=600 , **y_cols):
    fig = sp.make_subplots(specs=[[{"secondary_y": True}]])

    for i, (y_col, y_col_name) in enumerate(y_cols.items()):
        color = custom_colors[i % len(custom_colors)]
        trace = go.Scatter(x=x_col, y=df[y_col], mode='lines', name=y_col_name, line=dict(color=color))
        secondary_y = i > 0
        
        fig.add_trace(trace, secondary_y=secondary_y)
        if secondary_y:
            # 配置次y轴
            axis_name = f"yaxis{i + 1}"
            fig.update_layout(
                **{axis_name: dict(
                title=y_col_name,
                titlefont=dict(color=color),
                tickfont=dict(color=color),
                anchor="x",
                overlaying="y",
                side="right",
                position=1 - i * 0.05,
                showgrid=False
            )})
        else:
            # 配置主要y轴
            fig.update_yaxes(title_text=y_col_name, showgrid=False)

    # 配置x轴
    # fig.update_xaxes(title_text=str(x_col), showgrid=False)

    # 设置图表的布局
    fig.update_layout(
        title_text='和'.join(y_cols.values()),
        autosize=False,
        width=width,
        height=height,
        title_x=0.1,
        # plot_bgcolor='white',
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(0,0,0,0)",
        xaxis=dict(showgrid=False),
        font=dict(
            family="Courier New, monospace",
            size=30,
            color="black"
        )
    )
    fig.update_yaxes(showline=True, linewidth=2, linecolor='rgba(0,0,0,0)', mirror=True)

    # fig.show()
    return fig

plot_lines(df=display_df, x_col=display_df.index,width=1600,height=600, soc='电量', volt='电压')


In [11]:
# df_list.to_csv('all_battery.csv', index=False)

In [5]:
numeric_col=df_list.select_dtypes(include=[np.number]).columns
df_list[numeric_col]=df_list[numeric_col].astype('float32')
df_list['label'] = df_list['label'].astype('int8')
df_list['sn'] = df_list['sn'].astype('int32')
ic(numeric_col)
df_list.info()
df_list.head()

ic| numeric_col: Index(['volt', 'current', 'soc', 'max_single_volt', 'min_single_volt',
                        'max_temp', 'min_temp', 'timestamp', 'label', 'sn'],
                       dtype='object')


<class 'pandas.core.frame.DataFrame'>
Index: 7267584 entries, 0 to 255
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   volt             float32
 1   current          float32
 2   soc              float32
 3   max_single_volt  float32
 4   min_single_volt  float32
 5   max_temp         float32
 6   min_temp         float32
 7   timestamp        float32
 8   label            int8   
 9   sn               int32  
dtypes: float32(8), int32(1), int8(1)
memory usage: 311.9 MB


,volt,current,soc,max_single_volt,min_single_volt,max_temp,min_temp,timestamp,label,sn
0,152.100006,-41.700001,20.299999,1.695,1.687,162.0,156.0,906.0,0,0
1,152.100006,-41.700001,20.299999,1.696,1.687,162.0,156.0,908.0,0,0
2,152.100006,-41.700001,20.299999,1.696,1.687,162.0,156.0,909.0,0,0
3,152.100006,-41.700001,20.299999,1.695,1.687,162.0,156.0,910.0,0,0
4,152.100006,-41.700001,20.299999,1.696,1.687,162.0,156.0,911.0,0,0


In [6]:
# df_list.groupby('label').apply(lambda x: x.sort_values(['sn', 'timestamp']))

df_last = df_list.groupby('label').apply(lambda x: x.sort_values(['sn', 'timestamp'])).groupby('sn').last()
df_last.shape

(28389, 9)

In [7]:
# df_last.to_csv('df_last.csv', index=False)

In [8]:
# random.shuffle(df_list)

# 赋值数据集
df = df_last
df = df.reset_index(drop=True)

# df.head()


In [9]:

# 将数据和标签分离出来
X_list = df.drop(['label','timestamp'], axis=1)
y_list = df['label']
ic(X_list.shape)
X_list.head()


ic| X_list.shape: (28389, 7)


,volt,current,soc,max_single_volt,min_single_volt,max_temp,min_temp
0,153.800003,-41.700001,26.700001,1.716,1.706,180.0,168.0
1,150.100006,-4.100000,36.200001,1.670,1.662,192.0,180.0
2,149.100006,-5.100000,26.700001,1.658,1.654,192.0,186.0
3,163.399994,-4.500000,56.500000,1.819,1.811,198.0,180.0
4,159.899994,-32.099998,47.599998,1.781,1.772,180.0,174.0


In [10]:
n_splits = 5
kf = KFold(n_splits=n_splits)  # 5 折交叉验证

accuracies = []

for train_index, test_index in kf.split(X=X_list, y=y_list):
    ic("TRAIN:", train_index, "TEST:", test_index)

    X_train, X_test = X_list[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = CatBoostClassifier(verbose=0, n_estimators=100)
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    accuracies.append(accuracy)

# 计算平均准确率
average_accuracy = np.mean(accuracies)
print(f'Average accuracy: {average_accuracy}')

ic| "TRAIN:": 'TRAIN:'
    train_index: array([ 5678,  5679,  5680, ..., 28386, 28387, 28388])
    "TEST:": 'TEST:'
    test_index: array([   0,    1,    2, ..., 5675, 5676, 5677])


KeyError: "None of [Index([ 5678,  5679,  5680,  5681,  5682,  5683,  5684,  5685,  5686,  5687,\n       ...\n       28379, 28380, 28381, 28382, 28383, 28384, 28385, 28386, 28387, 28388],\n      dtype='int64', length=22711)] are in the [columns]"

# 切分数据集

In [10]:
test_size=0.3
# 切分数据
X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, test_size=test_size, random_state=2024)


# ic(X_train.shape)
# ic(X_test.shape)
# ic(y_train.shape)
# ic(y_test.shape);

In [11]:
# K 折交叉验证

In [19]:

# 初始化 KFold 类
from sklearn.model_selection import KFold
n_splits = 5

kf = KFold(n_splits=n_splits)  # 5 折交叉验证

accuracies = []

for train_index, test_index in kf.split(X_list):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    clf = CatBoostClassifier(verbose=0, n_estimators=100)
    clf.fit(X_train, y_train)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    accuracies.append(accuracy)

# 计算平均准确率
average_accuracy = np.mean(accuracies)
print(f'Average accuracy: {average_accuracy}')

NameError: name 'X' is not defined

In [12]:

import xgboost as xgb
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'n_estimators': 500,
    'max_depth': 8,
    'learning_rate': 0.01,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0,
    'reg_alpha': 0,
    'reg_lambda': 1,
    'seed': 7,
    'nthread': -1,
    'use_label_encoder': False
}

clf = xgb.XGBClassifier(
            **xgb_params
        )

clf.fit(X_train, y_train)



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None, nthread=-1,
              num_parallel_tree=None, ...)

In [22]:
# import lightgbm as lgb
# params = {'num_leaves': 60, #结果对最终效果影响较大，越大值越好，太大会出现过拟合
#           'min_data_in_leaf': 30,
#           'objective': 'binary', #定义的目标函数
#           'max_depth': 5,
#           'learning_rate': 0.03,
#           "min_sum_hessian_in_leaf": 6,
#           "boosting": "gbdt",
#           "feature_fraction": 0.9,  #提取的特征比率
#           "bagging_freq": 1,
#           "bagging_fraction": 0.8,
#           "bagging_seed": 11,
#           "lambda_l1": 0.1,             #l1正则
#           'lambda_l2': 0.001,     #l2正则
#           "verbosity": -1,
#           "nthread": -1,                #线程数量，-1表示全部线程，线程越多，运行的速度越快
#           'metric': {'binary_logloss', 'auc'},  ##评价函数选择
#           "random_state": 2019, #随机数种子，可以防止每次运行的结果不一致
#           # 'device': 'gpu' ##如果安装的事gpu版本的lightgbm,可以加快运算
#           }
# clf = lgb.LGBMClassifier()
# clf.fit(X_train, y_train)

In [23]:
# from catboost import CatBoostClassifier
# clf = CatBoostClassifier(verbose=0, n_estimators=100)
# clf.fit(X_train, y_train)


In [13]:
y_pred = clf.predict_proba(X_test)

def adjust_threshold(probs, threshold):
    # 提取预测为正类的概率（对于二分类问题，通常是第二列）
    positive_probs = probs[:, 1]

    # 根据阈值生成新的预测结果
    y_pred = [1 if prob >= threshold else 0 for prob in positive_probs]

    return y_pred
    
y_pred = adjust_threshold(y_pred, 0.5)
# sum(y_pred)


In [14]:
from src.model_evaluator import ModelEvaluator


res_eva = ModelEvaluator.evaluate(y_test, y_pred)
matrix = confusion_matrix(y_test, y_pred)

ModelEvaluator.draw_confusion_matrix(c_matrix=matrix, classes = ['正常', '损坏'])


AUC: 0.845523
Accuracy: 0.934601
Precision: 0.870354
Recall: 0.712164
Specificity: 0.9788821624665635
Confusion Matrix: 
[[6953  150]
 [ 407 1007]]


# mlflow log

In [91]:
print(os.environ["AWS_ACCESS_KEY_ID"])
print(os.environ["AWS_SECRET_ACCESS_KEY"])

minio
minio123


In [108]:
import boto3
from botocore.client import Config

# MinIO 服务器信息
minio_endpoint = 'http://192.168.31.25:9000'
minio_access_key = 'minio'
minio_secret_key = 'minio123'
bucket_name = 'mlflow'
file_path = './ml.ipynb'
object_name = 'file.txt'  # 上传到 MinIO 中的对象名

# 创建 S3 客户端
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=minio_access_key,
    aws_secret_access_key=minio_secret_key,
    config=Config(signature_version='s3v4'),
    region_name='us-east-1'
)

# 创建 bucket（如果不存在）
try:
    s3_client.head_bucket(Bucket=bucket_name)
except:
    s3_client.create_bucket(Bucket=bucket_name)

# 上传文件
s3_client.upload_file(file_path, bucket_name, object_name)

print(f"File {file_path} uploaded to {bucket_name}/{object_name}")


File ./ml.ipynb uploaded to mlflow/file.txt


In [103]:
import requests
response = requests.get("http://192.168.31.25:9001")
print(response.status_code)

200


In [107]:
print(mlflow.get_artifact_uri())



2024/05/19 15:25:16 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/05/19 15:25:16 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


s3://mlflow/2/a3f9429cd2fe4915a7b3df7874c1a5dc/artifacts


In [72]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
# mlflow.set_tracking_uri(uri="./mlflow_test/")
# os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://127.0.0.1:9000'
# os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'minio123'
from mlflow.models import infer_signature
# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Qqqckstart")
import os


mlflow.end_run()

# Start an MLflow run
with mlflow.start_run() as run:
    ic(run.info.run_id)

    # Log the hyperparameters
    mlflow.log_params(xgb_params)
    # Log the loss metric
    mlflow.log_metrics(res_eva)
    # mlflow.log_metric("precision", res_eva["precision"])
    # mlflow.log_metric("recall", res_eva["recall"])
    # mlflow.log_metric("specificity", res_eva["specificity"])
    import psutil

    # mlflow.log_metric("accuracy", res_eva["accuracy"])
    # cpu_percent = psutil.cpu_percent()
    # virtual_memory = psutil.virtual_memory().percent
    mlflow.system_metrics.enable_system_metrics_logging()
    time.sleep(5)
    # mlflow.log_("cpu_percent", cpu_percent)
    # mlflow.log_metric("virtual_memory_percent", virtual_memory) 
    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("问题定义", "时间序列")
    mlflow.set_tag("模型", "树模型")
    # Infer the model signature
    signature = infer_signature(X_train, clf.predict(X_train))
    # os.environ['AWS_ACCESS_KEY_ID'] = 'Z0qLwDccsRYWpKNadFYo'
    # os.environ['AWS_SECRET_ACCESS_KEY'] = '26QXPzDoLDm7kVsszwizjNN1AMjudl5Il7oNQuw6'
    # os.environ['MLFLOW_S3_ENDPOINT_URL'] = 'http://192.168.31.25:9000'
    # os.environ['AWS_MAX_ATTEMPTS'] = '1'
    mlflow.log_artifact('./ml.ipynb')
    model_info = mlflow.sklearn.log_model(
        sk_model=clf,
        artifact_path="mlflow/mlflow",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
        
    )

2024/05/19 17:17:32 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/05/19 17:17:32 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
ic| run.info.run_id: '7f055ccb9de44ad3beac98861c5015a9'
Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/05/19 17:17:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 4
Created version '4' of model 'tracking-quickstart'.
2024/05/19 17:17:46 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/05/19 17:17:46 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [85]:
# 部署模型
import mlflow.pyfunc

# Specify the run_id and artifact path
run_id = "7f055ccb9de44ad3beac98861c5015a9"
artifact_path = "mlflow/mlflow"

# Load the model
model_uri = f"runs:/{run_id}/{artifact_path}"
model = mlflow.pyfunc.load_model(model_uri)
# Now you can use the model object to make predictions
# For example:
# Assuming X_test is a pandas DataFrame
X_test.columns = ['f{}'.format(i) for i in range(len(X_test.columns))]


# Now you can use the model object to make predictions
predictions = model.predict(X_test.iloc[[20]])
predictions
X_test.iloc[20].to_frame().T


2024/05/19 17:20:52 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


array([1])

In [87]:
# 模型部署成 flask 服务
model.serve(port=5000)


AttributeError: 'PyFuncModel' object has no attribute 'serve'

In [89]:
import mlflow.data
import pandas as pd
from mlflow.data.pandas_dataset import PandasDataset


dataset_source_url = "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv"
raw_data = pd.read_csv(dataset_source_url, delimiter=";")
raw_data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [90]:

# Create an instance of a PandasDataset
dataset = mlflow.data.from_pandas(
    raw_data, source=dataset_source_url, name="wine quality - white", targets="quality"
)

# 保存onnx 模型

In [40]:
import onnxmltools
from onnxmltools.convert import convert_xgboost, convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# Convert XGBoost model to ONNX
onnx_model = convert_xgboost(clf, initial_types=initial_type)

# Log model in ONNX format
with mlflow.start_run():
    mlflow.onnx.log_model(onnx_model, "model")

2024/05/19 16:43:52 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/05/19 16:43:52 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2024/05/19 16:43:56 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/05/19 16:43:56 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


# 加载Onnx

In [44]:
import mlflow.onnx

# Specify the run_id and artifact path
run_id = run.info.run_id
artifact_path = "model"

# Load the ONNX model
model_uri = f"runs:/{run_id}/{artifact_path}"
onnx_model = mlflow.onnx.load_model(model_uri)

OSError: No such file or directory: '/Users/acke/JetBrainProject/trina-Projects/battery/mlflow_test/411181301438985947/1cc65e63979544d89920f143c2cc499b/artifacts/model'

In [38]:
# Assuming X_train is your training data
# 更改列名为 xgb 格式??? 失去列名
new_feature_names = ['f{}'.format(i) for i in range(X_train.shape[1])]
X_train.columns = new_feature_names

# Train your XGBoost model
clf.fit(X_train, y_train)

# Now you can convert your model to ONNX format
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
onnx_model = convert_xgboost(clf, initial_types=initial_type)

# 开始一个MLflow运行
with mlflow.start_run():
    # 保存并注册ONNX模型
    mlflow.onnx.log_model(onnx_model, "model", registered_model_name="my_onnx_model")

2024/05/19 16:19:36 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/05/19 16:19:36 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
Successfully registered model 'my_onnx_model'.
2024/05/19 16:19:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: my_onnx_model, version 1
Created version '1' of model 'my_onnx_model'.
2024/05/19 16:19:42 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/05/19 16:19:42 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [33]:
# 注册模型
import mlflow

# 假设你已经在一个MLflow运行中保存了一个模型
run_id = run.info.run_id
model_uri = "runs:/" + run_id + "/model"

# 注册模型
mlflow.register_model(
    model_uri,
    "my_model"
)

Successfully registered model 'my_model'.
2024/05/19 16:16:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: my_model, version 1
Created version '1' of model 'my_model'.


<ModelVersion: aliases=[], creation_timestamp=1716106604492, current_stage='None', description='', last_updated_timestamp=1716106604492, name='my_model', run_id='7241d6412f4a480398237a5c576356c4', run_link='', source='s3://mlflow/2/7241d6412f4a480398237a5c576356c4/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

2024/05/19 15:53:09 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: `pynvml` is not installed, to log GPU metrics please run `pip install pynvml` to install it..
2024/05/19 15:53:09 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.
2024/05/19 15:53:24 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/05/19 15:53:24 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


<RunData: metrics={'system/cpu_utilization_percentage': 30.4,
 'system/disk_available_megabytes': 41627.8,
 'system/disk_usage_megabytes': 10246.7,
 'system/disk_usage_percentage': 19.8,
 'system/network_receive_megabytes': 0.0010240000001431326,
 'system/network_transmit_megabytes': 0.0010240000001431326,
 'system/system_memory_usage_megabytes': 6373.0,
 'system/system_memory_usage_percentage': 37.1}, params={}, tags={'mlflow.runName': 'resilient-sow-3',
 'mlflow.source.name': '/Users/acke/Library/Caches/pypoetry/virtualenvs/battery-j7E_kv4E-py3.10/lib/python3.10/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'acke'}>


In [17]:
# onnx保存模型
import mlflow.onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Convert sklearn XGBoost model to ONNX
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]
onnx_model = convert_sklearn(clf, initial_types=initial_type)

# Log model in ONNX format
with mlflow.start_run():
    mlflow.onnx.log_model(onnx_model, "model")

# Load model in ONNX format
loaded_model = mlflow.onnx.load_model("runs:/<run_id>/model")

MissingShapeCalculator: Unable to find a shape calculator for type '<class 'xgboost.sklearn.XGBClassifier'>'.
It usually means the pipeline being converted contains a
transformer or a predictor with no corresponding converter
implemented in sklearn-onnx. If the converted is implemented
in another library, you need to register
the converted so that it can be used by sklearn-onnx (function
update_registered_converter). If the model is not yet covered
by sklearn-onnx, you may raise an issue to
https://github.com/onnx/sklearn-onnx/issues
to get the converter implemented or even contribute to the
project. If the model is a custom model, a new converter must
be implemented. Examples can be found in the gallery.


# 加载模型

In [119]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
ic(model_info)
ic(model_info.model_uri)

ic| model_info: <mlflow.models.model.ModelInfo object at 0x337c95300>
ic| model_info.model_uri: 'runs:/6f6e76e7de134bc6ba59a7b2dd9996b7/mlflow/mlflow'


'runs:/6f6e76e7de134bc6ba59a7b2dd9996b7/mlflow/mlflow'

In [126]:
predictions = loaded_model.predict(X_test)

iris_feature_names = X_train.columns

# Convert X_test validation feature data to a Pandas DataFrame
result = pd.DataFrame(X_test, columns=iris_feature_names)

# Add the actual classes to the DataFrame
result["actual_class"] = y_test

# Add the model predictions to the DataFrame
result["predicted_class"] = predictions


AUC: 0.845523
Accuracy: 0.934601
Precision: 0.870354
Recall: 0.712164
Specificity: 0.9788821624665635
Confusion Matrix: 
[[6953  150]
 [ 407 1007]]


,volt,current,soc,max_single_volt,min_single_volt,max_temp,min_temp,actual_class,predicted_class
17835,155.2,-5.0,33.7,1.726,1.714,120.0,108.0,0,0
10170,151.8,-18.2,24.1,1.689,1.680,216.0,204.0,0,0
2568,148.8,-5.1,25.4,1.655,1.651,264.0,234.0,1,1
6260,158.7,-27.5,47.6,1.768,1.761,294.0,282.0,0,1
2294,156.1,-5.1,45.1,1.736,1.724,90.0,78.0,0,0
6188,147.7,-3.7,21.6,1.643,1.640,192.0,180.0,0,0
24040,165.5,-4.5,59.1,1.843,1.835,168.0,150.0,1,1
13038,161.4,-3.9,54.0,1.796,1.785,114.0,96.0,0,0
11838,151.3,-4.8,36.8,1.683,1.675,192.0,156.0,0,0
14828,155.0,-5.0,41.3,1.723,1.712,138.0,126.0,1,0


In [ ]:
version: "3.3" services: db: restart: always image: postgres container_name: mlflow_db expose: - "${PG_PORT}" networks: - backend environment: - POSTGRES_USER=${PG_USER} - POSTGRES_PASSWORD=${PG_PASSWORD} - POSTGRES_DATABASE=${PG_DATABASE} volumes: - ./db_data:/var/lib/postgresql/data/ healthcheck: test: ["CMD", "pg_isready", "-p", "${PG_PORT}", "-U", "${PG_USER}"] interval: 5s timeout: 5s retries: 3 s3: restart: always image: minio/minio container_name: mlflow_minio volumes: - ./minio_data:/data ports: - "${MINIO_PORT}:9000" - "${MINIO_CONSOLE_PORT}:9001" networks: - frontend - backend environment: - MINIO_ROOT_USER=${MINIO_ROOT_USER} - MINIO_ROOT_PASSWORD=${MINIO_ROOT_PASSWORD} - MINIO_ADDRESS=${MINIO_ADDRESS} - MINIO_PORT=${MINIO_PORT} - MINIO_STORAGE_USE_HTTPS=${MINIO_STORAGE_USE_HTTPS} - MINIO_CONSOLE_ADDRESS=${MINIO_CONSOLE_ADDRESS} command: server /data healthcheck: test: ["CMD", "curl", "-f", "http://localhost:9000/minio/health/live"] interval: 30s timeout: 20s retries: 3 tracking_server: restart: always build: ./mlflow image: mlflow_server container_name: mlflow_server depends_on: - db ports: - "${MLFLOW_PORT}:5000" networks: - frontend - backend environment: - AWS_ACCESS_KEY_ID=${MINIO_ACCESS_KEY} - AWS_SECRET_ACCESS_KEY=${MINIO_SECRET_ACCESS_KEY} - MLFLOW_S3_ENDPOINT_URL=http://s3:${MINIO_PORT} - MLFLOW_S3_IGNORE_TLS=true command: > mlflow server --backend-store-uri postgresql://${PG_USER}:${PG_PASSWORD}@db:${PG_PORT}/${PG_DATABASE} --host 0.0.0.0 --serve-artifacts --artifacts-destination s3://${MLFLOW_BUCKET_NAME} healthcheck: test: ["CMD", "curl", "-f", "http://localhost:${MLFLOW_PORT}/"] interval: 30s timeout: 10s retries: 3 volumes: db_data: minio_data: networks: frontend: driver: bridge backend: driver: bridge
